In [ ]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
#model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def fillInTheBlanks(sentence, topK):
  #Convert to BERT format
  #NOTE: This must be done for every sentence. As an assignment, you can try to change this to reflect multiple sentences.
  text = "[CLS]" + sentence + "[SEP]"
  tokenizedText = tokenizer.tokenize(text)
  maskedIndex = tokenizedText.index("[MASK]")
  indexedTokens = tokenizer.convert_tokens_to_ids(tokenizedText)
  tokensTensor = torch.tensor([indexedTokens])
  #print(tokensTensor)
  with torch.no_grad():
    outputs = model(tokensTensor)
    predictions = outputs[0]
    #print(predictions)

  probabilities = torch.nn.functional.softmax(predictions[0, maskedIndex], dim=-1)
  topKWeights, topKIndices = torch.topk(probabilities, topK, sorted=True)

  for i, predictionIndex in enumerate(topKIndices):
    predictedToken = tokenizer.convert_ids_to_tokens([predictionIndex])[0]
    tokenWeight = topKWeights[i]
    print("[MASK]: ", predictedToken, "| weight = ", float(tokenWeight))

In [ ]:
fillInTheBlanks("The sky is [MASK] and the sea is [MASK].", topK=2)

[MASK]:  blue | weight =  0.36422455310821533
[MASK]:  clear | weight =  0.18508043885231018
